In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from comet_ml import Experiment
import numpy as np
import scipy.spatial
import pandas as pd
import comet_ml
import sklearn.decomposition
import matplotlib.pyplot as plt
# import keras
from sklearn import preprocessing
from sklearn.metrics import pairwise_distances,mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from utils.readProfiles import readMergedProfiles,readMergedProfiles2
from utils.pred_models import *
from utils.saveAsNewSheetToExistingFile import saveAsNewSheetToExistingFile

# from utils import networksEvol, tsne, readProfiles
import umap

#### In this notebook we test how combinting two data modalities will improve preformance of the following tasks:
    1 - MOA prediction
    2 - ?

## MOA Prediction:
- Methods:
    - Baseline: simple concatenation
    - SNF
    - prbabilistic modeling

In [22]:
# list(set(mergProf_treatLevel.columns.tolist())-set(l1k_features))
import os
os.listdir('./preprocessed_data/LINCS-Pilot1/CellPainting/')

['replicate_level_cp_normalized_dmso.csv.gz',
 'replicate_level_cp_augmented.csv.gz',
 'replicate_level_cp_normalized_feature_select_dmso.csv.gz',
 'replicate_level_cp_normalized_feature_select.csv.gz',
 'replicate_level_cp_normalized.csv.gz']

## Treatment level

In [2]:
dataset_rootDir='./';pertColName='PERT'

# dataset options: 'CDRP' , 'LUAD', 'TAORF', 'LINCS'
dataset='LINCS';

# CP Profile Type options: 'augmented' , 'normalized', 'normalized_variable_selected'
# lincs --> normalized_feature_select_dmso
profileType='normalized_feature_select_dmso'
profileLevel='treatment'; #'replicate'  or  'treatment'

# n of samples for replicate picking options: numbers or, 'max'
nRep=8
mergProf_repLevel,mergProf_treatLevel,cp_features,l1k_features=\
readMergedProfiles(dataset_rootDir,dataset,profileType,profileLevel,nRep);
# mergProf_repLevel,mergProf_treatLevel,l1k_features,cp_features,pertColName=readMergedProfiles(dataset,profileType,nRep)
# cp_features,l1k_features=cp_features.tolist(),l1k_features.tolist()


if profileLevel=='replicate':
    l1k=mergProf_repLevel[[pertColName]+l1k_features]
    cp=mergProf_repLevel[[pertColName]+cp_features]
elif profileLevel=='treatment':
    l1k=mergProf_treatLevel[list(set(mergProf_treatLevel.columns.tolist())-set(cp_features))]
    cp=mergProf_treatLevel[list(set(mergProf_treatLevel.columns.tolist())-set(l1k_features))]


scaler_ge = preprocessing.StandardScaler()
scaler_cp = preprocessing.StandardScaler()
l1k_scaled=l1k.copy()
l1k_scaled[l1k_features] = scaler_ge.fit_transform(l1k[l1k_features].values)
cp_scaled=cp.copy()
cp_scaled[cp_features] = scaler_cp.fit_transform(cp[cp_features].values.astype('float64'))

if 1:
    cp=cp_scaled.copy()
    l1k=l1k_scaled.copy()

/home/ubuntu/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (18,19,1249,1250) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


LINCS: Replicate Level Shapes (nSamples x nFeatures): cp:  52223 , 119 ,  l1k:  27837 , 978
l1k n of rep:  3.0
cp n of rep:  5.0
CP: from  9394  to  4647
l1k: from  8369  to  2338
CP and l1k high rep overlap:  1140
Treatment Level Shapes (nSamples x nFeatures+metadata): (1141, 122) (1141, 980) Merged Profiles Shape: (1141, 1101)


In [7]:
# LINCS: Replicate Level Shapes (nSamples x nFeatures): cp:  52223 , 1670 ,  l1k:  27837 , 978
# l1k n of rep:  3.0
# cp n of rep:  5.0
# CP: from  9394  to  4647
# l1k: from  8369  to  2338
# CP and l1k high rep overlap:  1140
# Treatment Level Shapes (nSamples x nFeatures+metadata): (1141, 1671) (1141, 979) Merged Profiles Shape: (1141, 2649)
mergProf_treatLevel.groupby(['Metadata_moa']).size()

Metadata_moa
11-beta hydroxysteroid dehydrogenase inhibitor    1
5 alpha reductase inhibitor                       2
ACAT inhibitor                                    4
AKT inhibitor                                     7
AKT inhibitor|TRAIL modulator                     2
                                                 ..
tyrosine kinase inhibitor                         2
vasopressin receptor antagonist                   2
vitamin D receptor agonist                        2
voltage-gated sodium channel blocker              1
xanthine oxidase inhibitor                        1
Length: 280, dtype: int64

In [21]:
# mergProf_treatLevel['Metadata_moa']=mergProf_treatLevel['Metadata_moa'].str.lower()
nSamplesforEachMOAclass=mergProf_treatLevel.groupby(['Metadata_moa']).size().reset_index().rename(columns={0:'size'}).sort_values(by=['size'],ascending=False).reset_index(drop=True)
listOfSelectedMoAs=nSamplesforEachMOAclass[nSamplesforEachMOAclass['size']>1]['Metadata_moa'].tolist()
print(len(listOfSelectedMoAs))

179


In [13]:
mergProf_treatLevel.Metadata_moa.unique().shape

(281,)

### Single Modalities Classification performance

In [24]:
from sklearn.decomposition import PCA
# %matplotlib inline
# Dimension reduction and clustering libraries
import umap
# import hdbscan
import sklearn.cluster as cluster
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
from sklearn.model_selection import LeaveOneOut,cross_val_score
from sklearn.linear_model import LogisticRegression
# nSamplesMOA=10 # results in grant for CDRP are for this number of MOAs
nSamplesMOA=1
# from MulticoreTSNE import MulticoreTSNE as TSNE
# df_1 = df_1.interpolate()
#############

##########
# mergProf_treatLevel['Metadata_moa']=mergProf_treatLevel['Metadata_moa'].str.lower()
mergProf_treatLevel=mergProf_treatLevel[~mergProf_treatLevel['Metadata_moa'].isnull()].reset_index(drop=True)
nSamplesforEachMOAclass=mergProf_treatLevel.groupby(['Metadata_moa']).size().reset_index().rename(columns={0:'size'}).sort_values(by=['size'],ascending=False).reset_index(drop=True)
listOfSelectedMoAs=nSamplesforEachMOAclass[nSamplesforEachMOAclass['size']>nSamplesMOA]['Metadata_moa'].tolist()
# corresPertID=[mergProf_treatLevel[mergProf_treatLevel['Metadata_moa']==i]['Metadata_pert_id'] for i in listOfSelectedMoAs]
# filteredMOAs=mergProf_treatLevel[mergProf_treatLevel['Metadata_moa'].isin(listOfSelectedMoAs)].reset_index(drop=True)
IDs4filteredMOAs=mergProf_treatLevel[mergProf_treatLevel['Metadata_moa'].isin(listOfSelectedMoAs)][pertColName].tolist()

data4eval=[[cp,cp_features],[l1k,l1k_features],[mergProf_treatLevel,cp_features+l1k_features]]

for r in range(len(data4eval)): #range(1):#
    print(r)
    domXdata=data4eval[r][0];
    domXfeats=data4eval[r][1]
    filteredMOAs=domXdata[domXdata[pertColName].isin(IDs4filteredMOAs)].reset_index(drop=True)

    data = filteredMOAs[domXfeats].values;
    labels=filteredMOAs.Metadata_moa.tolist()
        
    loocv = LeaveOneOut()
    model_loocv = LogisticRegression(multi_class='ovr',n_jobs=100,max_iter=1000)
    results_loocv = cross_val_score(model_loocv, data, labels, cv=loocv)
    print("Accuracy: %.2f%%" % (results_loocv.mean()*100.0))

0


KeyboardInterrupt: 

- CP
  - Accuracy: 66.83%
- L1k
  - Accuracy: 55.74%
- CP + L1k
  - Accuracy: 67.43%

In [37]:
filteredMOAs['Metadata_moa'].unique().shape

(179,)

In [ ]:
Clustering performance

In [29]:
from sklearn.decomposition import PCA
# %matplotlib inline
# Dimension reduction and clustering libraries
import umap
# import hdbscan
import sklearn.cluster as cluster
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
from sklearn.model_selection import LeaveOneOut,cross_val_score
from sklearn.linear_model import LogisticRegression
# nSamplesMOA=10 # results in grant for CDRP are for this number of MOAs
nSamplesMOA=1
# from MulticoreTSNE import MulticoreTSNE as TSNE
# df_1 = df_1.interpolate()
#############

##########
# mergProf_treatLevel['Metadata_moa']=mergProf_treatLevel['Metadata_moa'].str.lower()
mergProf_treatLevel=mergProf_treatLevel[~mergProf_treatLevel['Metadata_moa'].isnull()].reset_index(drop=True)
nSamplesforEachMOAclass=mergProf_treatLevel.groupby(['Metadata_moa']).size().reset_index().rename(columns={0:'size'}).sort_values(by=['size'],ascending=False).reset_index(drop=True)
listOfSelectedMoAs=nSamplesforEachMOAclass[nSamplesforEachMOAclass['size']>nSamplesMOA]['Metadata_moa'].tolist()
# corresPertID=[mergProf_treatLevel[mergProf_treatLevel['Metadata_moa']==i]['Metadata_pert_id'] for i in listOfSelectedMoAs]
# filteredMOAs=mergProf_treatLevel[mergProf_treatLevel['Metadata_moa'].isin(listOfSelectedMoAs)].reset_index(drop=True)
IDs4filteredMOAs=mergProf_treatLevel[mergProf_treatLevel['Metadata_moa'].isin(listOfSelectedMoAs)][pertColName].tolist()

data4eval=[[cp,cp_features],[l1k,l1k_features],[mergProf_treatLevel,cp_features+l1k_features]]

for r in range(len(data4eval)): #range(1):#
    print(r)
    domXdata=data4eval[r][0];
    domXfeats=data4eval[r][1]
    filteredMOAs=domXdata[domXdata[pertColName].isin(IDs4filteredMOAs)].reset_index(drop=True)

    data = filteredMOAs[domXfeats].values;
    labels=filteredMOAs.Metadata_moa.tolist()
        
    loocv = LeaveOneOut()
    model_loocv = LogisticRegression(multi_class='ovr',n_jobs=100,max_iter=1000)
    results_loocv = cross_val_score(model_loocv, data, labels, cv=loocv)
    print("Accuracy: %.2f%%" % (results_loocv.mean()*100.0))

0


KeyboardInterrupt: 

### SNF data fusion

In [39]:
from sklearn.cluster import SpectralClustering
filteredMOAs=mergProf_treatLevel[mergProf_treatLevel[pertColName].isin(IDs4filteredMOAs)].reset_index(drop=True)

snfInput=[filteredMOAs[cp_features].values,filteredMOAs[l1k_features].values]
# snfInput=[filteredMOAs[cp_features].values,filteredMOAs[cp_features].values]
# snfInput=[filteredMOAs[l1k_features].values,filteredMOAs[l1k_features].values]

from snf import compute
affinities = compute.make_affinity(snfInput, metric='euclidean')

# fuse the similarity matrices with SNF
fused = compute.snf(affinities)

# domXdata=mergProf_treatLevel;
# # domXfeats=data4eval[r][1]
# filteredMOAs=domXdata[domXdata[pertColName].isin(IDs4filteredMOAs)].reset_index(drop=True)

# data = fused[domXdata[domXdata[pertColName].isin(IDs4filteredMOAs)].index];
labels=filteredMOAs.Metadata_moa.tolist()

le_2 = preprocessing.LabelEncoder()
labels_categorical_2=filteredMOAs.Metadata_moa.unique().tolist();
le_2.fit(labels_categorical_2)
labels_numerical_2=le_2.transform(labels)


pred_labels = spectral_clustering(fused, n_clusters=179)

# pred_labels = SpectralClustering(n_clusters=179,assign_labels="discretize",random_state=0)\
#     .fit(filteredMOAs[cp_features].values).labels_


print('nmi: ',v_measure_score(pred_labels, labels))

# # accuracy_coclus(labels_numerical_2, pred_labels)

# cm = confusion_matrix(labels_numerical_2, pred_labels)
# print(cm.shape)
# # deprecated: indexes = linear_assignment(_make_cost_m(cm))
# indexes = linear_sum_assignment(_make_cost_m(cm))
# # print(indexes)
# total = 0
# # for row, column in indexes:

# for i in range(cm.shape[0]):
#     row, column=indexes[0][i],indexes[1][i]
#     value = cm[row][column]
#     total += value

# acc=(total * 1. / np.sum(cm))
# print(acc)
# # loocv = LeaveOneOut()
# # model_loocv = LogisticRegression(multi_class='ovr',n_jobs=100,max_iter=1000)
# # results_loocv = cross_val_score(model_loocv, data, labels, cv=loocv)
# # print("Accuracy: %.2f%%" % (results_loocv.mean()*100.0))

nmi:  0.7954179766995367


- CP
  - nmi: 0.78
- L1k
  - nmi: 0.73
- CP + L1k (snf)
  - nmi: 0.79

In [104]:
pred_labels.shape
clustering = SpectralClustering(n_clusters=2,
...         assign_labels="discretize",
...         random_state=0).fit(X)

(1010,)

In [12]:
from sklearn.cluster import spectral_clustering
from sklearn.metrics import v_measure_score, accuracy_score

pred_labels = spectral_clustering(fused, n_clusters=179)
v_measure_score(pred_labels, labels)

0.7849830085580078

In [79]:
v_measure_score(labels_numerical_2, pred_labels)

0.782523187741443

In [76]:
# len(indexes)
indexes[1].shape

(179,)

In [78]:
# from coclust.evaluation.external import accuracy
le_2 = preprocessing.LabelEncoder()
labels_categorical_2=filteredMOAs.Metadata_moa.unique().tolist();
le_2.fit(labels_categorical_2)
labels_numerical_2=le_2.transform(labels)

# accuracy_coclus(labels_numerical_2, pred_labels)

cm = confusion_matrix(labels_numerical_2, pred_labels)
print(cm.shape)
# deprecated: indexes = linear_assignment(_make_cost_m(cm))
indexes = linear_sum_assignment(_make_cost_m(cm))
# print(indexes)
total = 0
# for row, column in indexes:

for i in range(cm.shape[0]):
    row, column=indexes[0][i],indexes[1][i]
    value = cm[row][column]
    total += value

acc=(total * 1. / np.sum(cm))
print(acc)
# accuracy_score(pred_labels, labels)

(179, 179)
0.43267326732673267


In [91]:
# set(labels_numerical_2)

In [101]:
# labels_numerical_2.shape,pred_labels.shape
y_pred, confusion_matrix3 = get_y_preds(pred_labels, labels_numerical_2, 179);
accuracy_score(pred_labels, labels_numerical_2),accuracy_score(y_pred, labels_numerical_2)

(0.0009900990099009901, 0.43267326732673267)

In [100]:
# labels

In [95]:
v_measure_score(labels_numerical_2, y_pred)

0.7825231877414434

In [94]:
pred_labels

array([123,  48,  48, ...,  72,  92, 134], dtype=int32)

In [105]:
import numpy as np
from sklearn.metrics import confusion_matrix
# deprecated: from sklearn.utils.linear_assignment_ import linear_assignment
from scipy.optimize import linear_sum_assignment


def accuracy_coclus(true_row_labels, predicted_row_labels):
    """Get the best accuracy.
    Parameters
    ----------
    true_row_labels: array-like
        The true row labels, given as external information
    predicted_row_labels: array-like
        The row labels predicted by the model
    Returns
    -------
    float
        Best value of accuracy
    """

    cm = confusion_matrix(true_row_labels, predicted_row_labels)
    print(cm.shape)
    # deprecated: indexes = linear_assignment(_make_cost_m(cm))
    indexes = linear_sum_assignment(_make_cost_m(cm))
    print(indexes)
    total = 0
    for row, column in indexes:
        value = cm[row][column]
        total += value

    return (total * 1. / np.sum(cm))


def _make_cost_m(cm):
    s = np.max(cm)
    return (- cm + s)

In [63]:
# a=1
pred_labels.shape

(1010,)

In [41]:
len(set(labels))

179

In [36]:
domXdata=mergProf_treatLevel;
# domXfeats=data4eval[r][1]
fused[domXdata[domXdata[pertColName].isin(IDs4filteredMOAs)].index]

(1010, 1111)

In [28]:
domXdata[domXdata[pertColName].isin(IDs4filteredMOAs)].reset_index(drop=True).shape

(1010, 2652)

In [7]:
domXdata=mergProf_treatLevel;
# domXfeats=data4eval[r][1]
filteredMOAs=domXdata[domXdata[pertColName].isin(IDs4filteredMOAs)].reset_index(drop=True)

data = fused[domXdata[domXdata[pertColName].isin(IDs4filteredMOAs)].index];
labels=filteredMOAs.Metadata_moa.tolist()

loocv = LeaveOneOut()
model_loocv = LogisticRegression(multi_class='ovr',n_jobs=100,max_iter=1000)
results_loocv = cross_val_score(model_loocv, data, labels, cv=loocv)
print("Accuracy: %.2f%%" % (results_loocv.mean()*100.0))

['data', 'labels']

In [17]:
from snf import datasets
simdata = datasets.load_simdata()
# sorted(simdata.keys())
# ['data', 'labels']

# this dataset has two data arrays representing features from 200 samples
# >>> len(simdata.data)
# 2
# >>> len(simdata.labels)
# 200

# convert raw data arrays into sample x sample affinity matrices
from snf import compute
affinities = compute.make_affinity(simdata.data, metric='euclidean',K=20, mu=0.5)

# fuse the similarity matrices with SNF
fused = compute.snf(affinities,K=20)

# # estimate the number of clusters present in the fused matrix, derived via
# # an "eigengap" method (i.e., largest difference in eigenvalues of the
# # laplacian of the graph). note this function returns the top two options;
# # we'll only use the first
# first, second = compute.get_n_clusters(fused)
# # >>> first, second
# # (2, 5)

# # apply clustering procedure
# # you can use any clustering method here, but since SNF returns an affinity
# # matrix (i.e., all entries are positively-valued and indicate similarity)
# # spectral clustering makes a lot of sense
# >>> from sklearn import cluster
# >>> fused_labels = cluster.spectral_clustering(fused, n_clusters=first)

# # compute normalized mutual information for clustering solutions
# >>> from snf import metrics
# >>> labels = [simdata.labels, fused_labels]
# >>> for arr in affinities:
# ...     labels += [cluster.spectral_clustering(arr, n_clusters=first)]
# >>> nmi = metrics.nmi(labels)

# # compute silhouette score to assess goodness-of-fit for clustering
# >>> silhouette = metrics.silhouette_score(fused, fused_labels)

In [18]:
le_2 = preprocessing.LabelEncoder()
labels_categorical_2=filteredMOAs.Metadata_moa.unique().tolist();
le_2.fit(labels_categorical_2)
labels_numerical_2=le_2.transform(labels)

(200, 200)

In [14]:
y_pred, confusion_matrix = get_y_preds(cluster_assignments, y_true, n_clusters);

((200, 2), (200, 2))

In [ ]:
y_pred, confusion_matrix = get_y_preds(cluster_assignments, y_true, n_clusters);

In [89]:
from munkres import Munkres
def get_y_preds(cluster_assignments, y_true, n_clusters):
    '''
    Computes the predicted labels, where label assignments now
    correspond to the actual labels in y_true (as estimated by Munkres)

    cluster_assignments:    array of labels, outputted by kmeans
    y_true:                 true labels
    n_clusters:             number of clusters in the dataset

    returns:    a tuple containing the accuracy and confusion matrix,
                in that order
    '''
    confusion_matrix = sklearn.metrics.confusion_matrix(y_true, cluster_assignments, labels=None)
    # compute accuracy based on optimal 1:1 assignment of clusters to labels
    cost_matrix = calculate_cost_matrix(confusion_matrix, n_clusters)
    indices = Munkres().compute(cost_matrix)
    kmeans_to_true_cluster_labels = get_cluster_labels_from_indices(indices)
    y_pred = kmeans_to_true_cluster_labels[cluster_assignments]
    return y_pred, confusion_matrix 
def calculate_cost_matrix(C, n_clusters):
    cost_matrix = np.zeros((n_clusters, n_clusters))

    # cost_matrix[i,j] will be the cost of assigning cluster i to label j
    for j in range(n_clusters):
        s = np.sum(C[:,j]) # number of examples in cluster i
        for i in range(n_clusters):
            t = C[i,j]
            cost_matrix[j,i] = s-t
    return cost_matrix
def get_cluster_labels_from_indices(indices):
    n_clusters = len(indices)
    clusterLabels = np.zeros(n_clusters)
    for i in range(n_clusters):
        clusterLabels[i] = indices[i][1]
    return clusterLabels

In [ ]:
def AccMeasure(T,idx):
    # %Measure percentage of Accuracy and the Rand index of clustering results
    # % The number of class must equal to the number cluster 
    # %Output
    # % Acc = Accuracy of clustering results
    # % rand_index = Rand's Index,  measure an agreement of the clustering results
    # % match = 2xk mxtrix which are the best match of the Target and clustering results
    # %Input
    # % T = 1xn target index
    # % idx =1xn matrix of the clustering results
    # % EX:
    # % X=[randn(200,2);randn(200,2)+6,;[randn(200,1)+12,randn(200,1)]]; T=[ones(200,1);ones(200,1).*2;ones(200,1).*3];
    # % idx=kmeans(X,3,'emptyaction','singleton','Replicates',5);
    # %  [Acc,rand_index,match]=AccMeasure(T,idx)
    
    
    k=np.max([T.max(),idx.max()]);
    n=len(T);
    for i=1:k
        temp=find(T==i);
        a{i}=temp; #%#ok<AGROW>

    b1=[];
    t1=zeros(1,k);
    for i=1:k
        tt1=find(idx==i);
        for j=1:k
           t1(j)=sum(ismember(tt1,a{j}));
        
        b1=[b1;t1]; #%#ok<AGROW>

        Members=zeros(1,k); 

    P = perms((1:k));
        Acc1=0;
    for pi=1:size(P,1)
        for ki=1:k
            Members(ki)=b1(P(pi,ki),ki);
        
        if sum(Members)>Acc1
            match=P(pi,:);
            Acc1=sum(Members);
        
    
    rand_ss1=0;
    rand_dd1=0;
    for xi=1:n-1
        for xj=xi+1:n
            rand_ss1=rand_ss1+((idx(xi)==idx(xj))&&(T(xi)==T(xj)));
            rand_dd1=rand_dd1+((idx(xi)~=idx(xj))&&(T(xi)~=T(xj)));
        
    
    rand_index=200*(rand_ss1+rand_dd1)/(n*(n-1));
    Acc=Acc1/n*100; 
    match=[1:k;match];
    return [Acc,rand_index,match]

# Using MildInt

In [3]:
nSamplesMOA=5
# from MulticoreTSNE import MulticoreTSNE as TSNE
# df_1 = df_1.interpolate()
#############

##########
# mergProf_treatLevel['Metadata_moa']=mergProf_treatLevel['Metadata_moa'].str.lower()
mergProf_treatLevel=mergProf_treatLevel[~mergProf_treatLevel['Metadata_moa'].isnull()].reset_index(drop=True)
nSamplesforEachMOAclass=mergProf_treatLevel.groupby(['Metadata_moa']).size().reset_index().rename(columns={0:'size'}).sort_values(by=['size'],ascending=False).reset_index(drop=True)
listOfSelectedMoAs=nSamplesforEachMOAclass[nSamplesforEachMOAclass['size']>nSamplesMOA]['Metadata_moa'].tolist()
# corresPertID=[mergProf_treatLevel[mergProf_treatLevel['Metadata_moa']==i]['Metadata_pert_id'] for i in listOfSelectedMoAs]
# filteredMOAs=mergProf_treatLevel[mergProf_treatLevel['Metadata_moa'].isin(listOfSelectedMoAs)].reset_index(drop=True)
IDs4filteredMOAs=mergProf_treatLevel[mergProf_treatLevel['Metadata_moa'].isin(listOfSelectedMoAs)][pertColName].tolist()
    
filteredMOAs=mergProf_treatLevel[mergProf_treatLevel[pertColName].isin(IDs4filteredMOAs)].reset_index(drop=True)
labels=filteredMOAs.Metadata_moa.tolist()
print(filteredMOAs.shape)
le_2 = preprocessing.LabelEncoder()
labels_categorical_2=filteredMOAs.Metadata_moa.unique().tolist();
le_2.fit(labels_categorical_2)
labels_numerical_2=le_2.transform(labels)

#  Data m has a shape (#samples, length of time series, size of input dimension).
import sys
sys.path.insert(1, '../MildInt-master_v2')
from mmrnn import *

m = MMRNN()
cp_hidden=2
ge_hidden=2

# snfInput=[filteredMOAs[cp_features].values,filteredMOAs[l1k_features].values]
cp_m=filteredMOAs[cp_features].values[:,np.newaxis,:]
ge_m=filteredMOAs[l1k_features].values[:,np.newaxis,:]

m.append_component('cp', cp_m.shape[2], cp_hidden, cp_m.shape[1])
m.append_component('ge', ge_m.shape[2], ge_hidden, ge_m.shape[1])

IDs=pd.Series(list(range(cp_m.shape[0])))
m.append_data('cp', IDs, cp_m, labels_numerical_2, np.ones(cp_m.shape[0]))
m.append_data('ge', IDs, ge_m, labels_numerical_2, np.ones(cp_m.shape[0]))

## testIDs=np.array(range(cp_m.shape[1]))

(606, 1101)
Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [11]:
###m.append_test_overlapIDs(testIDs)
###m.append_training_overlapIDs(trainIDs)
# from DataManager import *
# dm = DataManager()
# 5-fold CV
overlapIDs=IDs.copy()
# test_folds = dm.generate_crossvalidation_set(IDs)
accuracy = []
for i in range(len(overlapIDs)):
    test_folds_IDs=[overlapIDs[i]]
    m.append_test_overlapIDs(pd.Series(test_folds_IDs))
#     trainIDs = overlapIDs[~overlapIDs.isin(test_folds[i])]
    trainIDs=list(set(overlapIDs)-set(test_folds_IDs))
    m.append_training_overlapIDs(pd.Series(trainIDs))

#     with tf.varialbe_scope('fold run'):
    m.build_integrative_network()
    m.training(len(trainIDs))
        
    accuracy.append(m.evaluate_accuracy())
#     tf.reset_default_graph()
    print(accuracy)
print(np.mean(accuracy))

/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


[9] [14]
[0.0]


/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


[9] [10]
[0.0, 0.0]


/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


...

/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


[44] [44]
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0]


/home/ubuntu/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


[44] [9]
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]


KeyboardInterrupt: 

In [4]:
# filteredMOAs[cp_features].values
# del m
# [overlapIDs[i]]
# cp_m.shape
# m.training(len(trainIDs))
# pd.Series(m.IDs['cp'])[~pd.Series(m.IDs['cp']).isin(m.test_overlapIDs)]

In [8]:
# m.IDs['cp']
# m.IDs['cp'][not (pd.Series(m.IDs['cp']).isin(m.test_overlapIDs).tolist())]
# len(overlapIDs)
# test_folds_IDs=[overlapIDs[0]]

In [7]:
overlapIDs=IDs.copy()
# test_folds = dm.generate_crossvalidation_set(IDs)
accuracy = []
for i in range(len(overlapIDs)):
    test_folds_IDs=[overlapIDs[i]]
    m.append_test_overlapIDs(pd.Series(test_folds_IDs))
#     trainIDs = overlapIDs[~overlapIDs.isin(test_folds[i])]
    trainIDs=list(set(overlapIDs)-set(test_folds_IDs))
    m.append_training_overlapIDs(pd.Series(trainIDs))

#     with tf.varialbe_scope('fold run'):
    m.build_integrative_network()
    m.training(len(trainIDs))
        
    accuracy.append(m.evaluate_accuracy())
#     tf.reset_default_graph()
    print(accuracy)
print(np.mean(accuracy))

[ 36  36  36 ... 154 154 154]
[36] [36]
[1.0]
[ 36  36  36 ... 154 154 154]
[36] [36]
[1.0, 1.0]
[ 36  36  36 ... 154 154 154]
[36] [36]
[1.0, 1.0, 1.0]
[ 36  36  36 ... 154 154 154]
[36] [82]
[1.0, 1.0, 1.0, 0.0]
[ 36  36  36 ... 154 154 154]
[36] [36]
[1.0, 1.0, 1.0, 0.0, 1.0]
[ 36  36  36 ... 154 154 154]
[174] [72]
[1.0, 1.0, 1.0, 0.0, 1.0, 0.0]
[ 36  36  36 ... 154 154 154]
[174] [59]
[1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0]
[ 36  36  36 ... 154 154 154]
[174] [53]
[1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[ 36  36  36 ... 154 154 154]
[174] [139]
[1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
[ 36  36  36 ... 154 154 154]
[174] [132]
[1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[ 36  36  36 ... 154 154 154]
[73] [163]
[1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[ 36  36  36 ... 154 154 154]
[156] [55]
[1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[ 36  36  36 ... 154 154 154]
[156] [82]
[1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[ 

KeyboardInterrupt: 

In [10]:
filteredMOAs.shape

(1010, 1101)

In [9]:
filteredMOAs.Metadata_moa.tolist()

['HSP inhibitor',
 'HSP inhibitor',
 'HSP inhibitor',
 'HSP inhibitor',
 'HSP inhibitor',
 'tubulin polymerization inhibitor',
 'tubulin polymerization inhibitor',
 'tubulin polymerization inhibitor',
 'tubulin polymerization inhibitor',
 'tubulin polymerization inhibitor',
 'androgen receptor modulator',
 'progesterone receptor agonist',
 'progesterone receptor agonist',
 'progesterone receptor agonist',
 'sodium channel blocker',
 'serotonin receptor antagonist',
 'adrenergic receptor antagonist',
 'glucocorticoid receptor agonist',
 'glucocorticoid receptor agonist',
 'glucocorticoid receptor agonist',
 'glucocorticoid receptor agonist',
 'glucocorticoid receptor agonist',
 'serotonin receptor agonist',
 'serotonin receptor agonist',
 'angiogenesis inhibitor|tumor necrosis factor production inhibitor',
 'angiogenesis inhibitor|tumor necrosis factor production inhibitor',
 'insulin sensitizer|PPAR receptor agonist',
 'glucocorticoid receptor agonist',
 'glucocorticoid receptor agonis